## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

from sklearn.model_selection import train_test_split

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [4]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [6]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.strip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ').replace('  ', ' ')
    return normed.strip()

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [7]:
# Answer below:
yelp['normalized'] = yelp['text'].apply(normalize)

In [8]:
yelp['normalized']

0                                         wow love place
1                                         crust not good
2                                 not tasti textur nasti
3      stop late may bank holiday rick steve recommen...
4                                select menu great price
                             ...                        
971                        think food flavor textur lack
972                               appetit instantli gone
973                 overal not impress would not go back
974    whole experi underwhelm think we ll go ninja s...
975    wast enough life pour salt wound draw time too...
Name: normalized, Length: 976, dtype: object

Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [9]:
# Answer below:
vocab_list = list(set([word for sent in [sent.split(' ') for sent in yelp['normalized']] for word in sent]))
vocab_size = len(vocab_list)
vocab_size

1629

In [10]:
encoded_sents = [one_hot(sent, vocab_size) for sent in yelp['normalized']]
encoded_sents[:5]

[[840, 436, 54],
 [1234, 1069, 1167],
 [1069, 1241, 774, 663],
 [1336, 879, 19, 625, 639, 1237, 949, 913, 436, 1020],
 [688, 375, 284, 1469]]

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [11]:
# Answer below:
ind_vars = pad_sequences(encoded_sents)
ind_vars[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  840,  436,   54],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [12]:
# Answer below:
X_train, X_test, y_train, y_test = train_test_split(ind_vars, yelp['sentiment'], test_size=0.2)

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((780, 83), (196, 83), (780,), (196,))

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [14]:
# Answer below:
max_words = np.max(ind_vars) #grabs largest numeric encoded value
max_words

1627

In [15]:
ind_vars.shape

(976, 83)

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length = ind_vars.shape[1])) # 100 is output of embedding layer
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 83, 100)           162900    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 167,429
Trainable params: 167,429
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [17]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=128,
          validation_data=(X_test, y_test))


Epoch 1/50
7/7 [==============================] - 2s 155ms/step - loss: 0.6960 - accuracy: 0.4672 - val_loss: 0.6937 - val_accuracy: 0.5408
Epoch 2/50
7/7 [==============================] - 1s 75ms/step - loss: 0.6651 - accuracy: 0.6707 - val_loss: 0.6751 - val_accuracy: 0.5561
Epoch 3/50
7/7 [==============================] - 0s 72ms/step - loss: 0.6223 - accuracy: 0.7642 - val_loss: 0.6685 - val_accuracy: 0.5867
Epoch 4/50
7/7 [==============================] - 0s 72ms/step - loss: 0.5893 - accuracy: 0.7887 - val_loss: 0.6591 - val_accuracy: 0.6173
Epoch 5/50
7/7 [==============================] - 1s 77ms/step - loss: 0.5324 - accuracy: 0.9041 - val_loss: 0.6453 - val_accuracy: 0.6684
Epoch 6/50
7/7 [==============================] - 1s 85ms/step - loss: 0.4819 - accuracy: 0.9256 - val_loss: 0.6259 - val_accuracy: 0.6888
Epoch 7/50
7/7 [==============================] - 0s 72ms/step - loss: 0.4202 - accuracy: 0.9524 - val_loss: 0.6038 - val_accuracy: 0.7245
Epoch 8/50
7/7 [==========

#Lecture Notes

In [18]:
from tensorflow.keras.preprocessing.text import one_hot

In [19]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']

In [20]:
target = [1,1,1,1,1,0,0,0,0,0]

In [21]:
vocab_size = 100
encoded_docs = [one_hot(doc, vocab_size) for doc in docs]

In [22]:
encoded_docs

[[32, 44],
 [67, 75],
 [53, 74],
 [77, 75],
 [29],
 [88],
 [67, 74],
 [68, 67],
 [67, 75],
 [18, 38, 44, 83]]

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
ind_vars = pad_sequences(encoded_docs)

In [25]:
ind_vars

array([[ 0,  0, 32, 44],
       [ 0,  0, 67, 75],
       [ 0,  0, 53, 74],
       [ 0,  0, 77, 75],
       [ 0,  0,  0, 29],
       [ 0,  0,  0, 88],
       [ 0,  0, 67, 74],
       [ 0,  0, 68, 67],
       [ 0,  0, 67, 75],
       [18, 38, 44, 83]], dtype=int32)

In [26]:
ind_vars.shape

(10, 4)

In [27]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

In [28]:
max_words = np.max(ind_vars) #grabs largest numeric encoded value
max_words

88

In [29]:
model = Sequential()
model.add(Embedding(max_words, 100, input_length = ind_vars.shape[1])) # 100 is output of embedding layer
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            8800      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 13,329
Trainable params: 13,329
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Fake data so doesn't work, but this is the main idea.
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(ind_vars, np.array(target), epochs=1, batch_size=1)